# 📘 Predicción de Series Temporales con Python
Este notebook muestra un flujo completo de análisis y predicción de series temporales.

In [ ]:
# =========================================================
# 1. Importación de librerías
# =========================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.arima.model import ARIMA

from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# =========================================================
# 2. Carga de datos
# =========================================================
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv'
df = pd.read_csv(url, parse_dates=['Month'], index_col='Month')
df.head()

In [ ]:
# =========================================================
# 3. Análisis Exploratorio
# =========================================================
plt.figure(figsize=(12,6))
plt.plot(df, label='Pasajeros')
plt.title('Serie Temporal de Pasajeros Aéreos')
plt.xlabel('Fecha')
plt.ylabel('Número de Pasajeros')
plt.legend()
plt.show()

result = seasonal_decompose(df['Passengers'], model='multiplicative')
result.plot()
plt.show()

In [ ]:
# =========================================================
# 4. Preparación de datos
# =========================================================
train = df.iloc[:-12]
test = df.iloc[-12:]
print(f'Tamaño train: {len(train)}')
print(f'Tamaño test: {len(test)}')

In [ ]:
# =========================================================
# 5. Modelos de Predicción
# =========================================================
model_hw = ExponentialSmoothing(train['Passengers'], seasonal='multiplicative', seasonal_periods=12).fit()
pred_hw = model_hw.forecast(12)

model_arima = ARIMA(train['Passengers'], order=(2,1,2))
model_arima_fit = model_arima.fit()
pred_arima = model_arima_fit.forecast(12)

In [ ]:
# =========================================================
# 6. Evaluación de Modelos
# =========================================================
def evaluate(y_true, y_pred, nombre):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    print(f'{nombre} -> MAE: {mae:.2f}, RMSE: {rmse:.2f}')

evaluate(test['Passengers'], pred_hw, 'Holt-Winters')
evaluate(test['Passengers'], pred_arima, 'ARIMA')

In [ ]:
# =========================================================
# 7. Visualización de Resultados
# =========================================================
plt.figure(figsize=(12,6))
plt.plot(train.index, train['Passengers'], label='Train')
plt.plot(test.index, test['Passengers'], label='Test', color='black')
plt.plot(test.index, pred_hw, label='Predicción Holt-Winters', color='blue')
plt.plot(test.index, pred_arima, label='Predicción ARIMA', color='red')
plt.title('Predicción de Pasajeros Aéreos')
plt.xlabel('Fecha')
plt.ylabel('Número de Pasajeros')
plt.legend()
plt.show()

In [ ]:
# =========================================================
# 8. Conclusiones
# =========================================================
print('''\nConclusiones:
- Holt-Winters capta mejor la estacionalidad de la serie.
- ARIMA también logra una predicción aceptable, pero depende de la elección del orden.
- Este cuaderno muestra el flujo completo: carga de datos, análisis exploratorio,
  descomposición de la serie, modelado y evaluación.
''')